In [ ]:
def 

In [4]:
IMG_CHANNELS = 1
IMG_ROWS = 64
IMG_COLS = 64

NUM_CLASSES = 2
NUM_SAMPLES = 447648
BATCH_SIZE = 32 # 한 epoch에서 실행시키는 단위(배치)크기
CACHE_BATCH_COUNT = 64

sample_bytes = IMG_CHANNELS * IMG_ROWS * IMG_COLS
batch_bytes = sample_bytes * BATCH_SIZE
cache_bytes = batch_bytes * CACHE_BATCH_COUNT

validation_num_samples = int(NUM_SAMPLES*1.0/4.0)
train_num_samples = NUM_SAMPLES - validation_num_samples

train_total_batch_count = train_num_samples / BATCH_SIZE
train_total_cache_count = (train_total_batch_count + (CACHE_BATCH_COUNT-1)) / CACHE_BATCH_COUNT

validation_total_batch_count = validation_num_samples / BATCH_SIZE
validation_total_cache_count = (validation_total_batch_count + (CACHE_BATCH_COUNT-1)) / CACHE_BATCH_COUNT

# index 기준은 batch 수
print('A : 한 배치 당 패치 수\t' + str(BATCH_SIZE))
print('B : 한 캐쉬 당 배치 수\t' + str(CACHE_BATCH_COUNT))
print('C = A x B : 한 캐쉬 당 패치 수\t' + str(BATCH_SIZE*CACHE_BATCH_COUNT))
print('D : 한 패치 당 바이트 수\t' + str(sample_bytes))
print('E = A x D : 한 배치 당 바이트 수\t' + str(batch_bytes))
print('F = E x B : 한 캐쉬 당 바이트 수\t' + str(cache_bytes) + ', ' + str(cache_bytes/1024) + 'KB')

# train
print('훈련 데이터 수\t' + str(train_num_samples))
print('훈련 배치 수\t' + str(train_total_batch_count))
print('훈련 캐쉬 수\t' + str(train_total_cache_count))

# validataion
print('검증 데이터 수\t' + str(validation_num_samples))
print('검증 배치 수\t' + str(validation_total_batch_count))
print('검증 캐쉬 수\t' + str(validation_total_cache_count))

A : 한 배치 당 패치 수	32
B : 한 캐쉬 당 배치 수	64
C = A x B : 한 캐쉬 당 패치 수	2048
D : 한 패치 당 바이트 수	4096
E = A x D : 한 배치 당 바이트 수	131072
F = E x B : 한 캐쉬 당 바이트 수	8388608, 8192KB
훈련 데이터 수	335736
훈련 배치 수	10491
훈련 캐쉬 수	164
검증 데이터 수	111912
검증 배치 수	3497
검증 캐쉬 수	55


In [7]:
def TrainDatasetGenerator():
    
    count = 0

    # x는 패치, y는 라벨
    fp_x = open('./dataset/train_image_64x64_gray_447648.bin', 'rb')
    fp_y = open('./dataset/train_label_64x64_gray_447648.bin', 'rb')

    offset_start_x = 0
    offset_start_y = 0
    
    # for문 구조
    # 전체 캐쉬수 만큼 루프
    for idx_cache in range(train_total_cache_count):

        # 해당 캐쉬 당 배치 수 계산 (마지막 캐쉬의 배치 수 계산 고려)
        if ((idx_cache + 1) * cache_batch_count) > train_total_batch_count:
            batch_count = train_total_batch_count - idx_cache * cache_batch_count
        else:
            batch_count = cache_batch_count

        # y인 경우에는 패치당 1바이트이므로 캐쉬당 패치수로 인덱스를 계산함            
        offset_x = offset_start_x + idx_cache * cache_bytes
        offset_y = offset_start_y + idx_cache * CACHE_BATCH_COUNT * BATCH_SIZE

        # 현재 캐쉬에서 읽어야 할 바이트 수는 배치 수 x 배치 바이트 만큼임
        read_cache_bytes = batch_bytes * batch_count

        fp_x.seek(offset_x, 1)
        fp_y.seek(offset_y, 1)

        # 패치 읽음
        # 파일에서 데이터를 읽어 캐쉬 버퍼에 로딩
        cache_buf_x = fp_x.read(read_cache_bytes)
        cache_data_x = np.frombuffer(cache_buf_x, dtype=np.uint8)
        cache_data_x = cache_data.reshape(batch_count, IMG_CHANNELS, IMG_ROWS, IMG_COLS)
        cache_data_x = cache_data.astype('float32')
        cache_data_x /= 255

        # 라벨 읽음
        cache_buf_y = fp_y.read(batch_count)
        cache_data_y = []
        for i in cache_buf_y:
            cache_data_y.append(i)
        cache_data_y = np.asarray(cache_data_y, dtype=np.uint8, order='C')
        # convert class vectors to binary class matrices
        cache_data_y = np_utils.to_categorical(cache_data_y, NUM_CLASSES)

        # 로딩한 캐쉬에서 배치 수 만큼 루프돌려 해당 배치를 넘겨줌
        for idx_batch in range(batch_count):

            print (count)
            count = count + 1

            yield cache_data_x[idx_batch*BATCH_SIZE:(idx_batch+1)*BATCH_SIZE], cache_data_y[idx_batch*BATCH_SIZE:(idx_batch+1)*BATCH_SIZE]

In [8]:
def ValidationDatasetGenerator():
    
    count = 0

    # x는 패치, y는 라벨
    fp_x = open('./dataset/train_image_64x64_gray_447648.bin', 'rb')
    fp_y = open('./dataset/train_label_64x64_gray_447648.bin', 'rb')

    offset_start_x = train_num_samples * sample_bytes
    offset_start_y = train_num_samples
    
    # for문 구조
    # 전체 캐쉬수 만큼 루프
    for idx_cache in range(train_total_cache_count):

        # 해당 캐쉬 당 배치 수 계산 (마지막 캐쉬의 배치 수 계산 고려)        
        if ((idx_cache + 1) * cache_batch_count) > train_total_batch_count:
            batch_count = train_total_batch_count - idx_cache * cache_batch_count
        else:
            batch_count = cache_batch_count

        # y인 경우에는 패치당 1바이트이므로 캐쉬당 패치수로 인덱스를 계산함            
        offset_x = offset_start_x + idx_cache * cache_bytes
        offset_y = offset_start_y + idx_cache * CACHE_BATCH_COUNT * BATCH_SIZE

        # 현재 캐쉬에서 읽어야 할 바이트 수는 배치 수 x 배치 바이트 만큼임        
        read_cache_bytes = batch_bytes * batch_count

        fp_x.seek(offset_x, 1)
        fp_y.seek(offset_y, 1)

        # 패치 읽음
        # 파일에서 데이터를 읽어 캐쉬 버퍼에 로딩
        cache_buf_x = fp_x.read(read_cache_bytes)
        cache_data_x = np.frombuffer(cache_buf_x, dtype=np.uint8)
        cache_data_x = cache_data.reshape(batch_count, IMG_CHANNELS, IMG_ROWS, IMG_COLS)
        cache_data_x = cache_data.astype('float32')
        cache_data_x /= 255

        # 라벨 읽음        
        cache_buf_y = fp_y.read(batch_count)
        cache_data_y = []
        for i in cache_buf_y:
            cache_data_y.append(i)
        cache_data_y = np.asarray(cache_data_y, dtype=np.uint8, order='C')
        # convert class vectors to binary class matrices
        cache_data_y = np_utils.to_categorical(cache_data_y, NUM_CLASSES)

        # 로딩한 캐쉬에서 배치 수 만큼 루프돌려 해당 배치를 넘겨줌        
        for idx_batch in range(batch_count):

            print (count)
            count = count + 1

            yield cache_data_x[idx_batch*BATCH_SIZE:(idx_batch+1)*BATCH_SIZE], cache_data_y[idx_batch*BATCH_SIZE:(idx_batch+1)*BATCH_SIZE]

In [ ]:
           
            buf = fp.read(file_size)
            
            for batch_it in range(readed_num_mem_batch)
                data = np.frombuffer(buf, dtype=np.uint8)
                data = data.reshape(count, channel, row, col)

            fp_it = fp_it + MEM_BUF_SIZE

    print('Loading data from', filepath)

    print('file size : ', os.path.getsize(filepath))
    print('calc size : ', count * channel * row * col)
    
    fp = open(filepath, 'rb')
    
    buf = fp.read(count * channel * row * col)
    data = np.frombuffer(buf, dtype=np.uint8)
    data = data.reshape(count, channel, row, col)

    print('loaded shape : ', data.shape)

    data = data.astype('float32')
    data /= 255

    return data
    
    

In [8]:
8794 / 128

68

In [6]:
536870912 / 1024 / 1024

512

In [9]:
68 * 128

8704

In [ ]:
while 1:
    for f_i in range()
    for i in range(20): # 1875 * 32 = 60000 -> # of training samples            
        print('i:' + str(i) + ' count:' + str(count))
        count = count + 1
        print('i*32:' + str(i*32) + '  (i+1)*32:' + str((i+1)*32))
        yield X_train[i*32:(i+1)*32], y_train[i*32:(i+1)*32] # 32만큼 전달한다.
    print("-------")

In [26]:
32 * 4

128

In [27]:
123*3

369

In [28]:
128*3

384

In [29]:
128*4

512

In [30]:
128*5

640

In [34]:
640 + 32 + 32 + 32 + 16

752